In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI (or Ollama)


In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print(
        "No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!"
    )
elif not api_key.startswith("sk-proj-"):
    print(
        "An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook"
    )
elif api_key.strip() != api_key:
    print(
        "An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook"
    )
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!


In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": message}]
)
print(response.choices[0].message.content)

Hello! It’s great to hear from you! How can I assist you today?


## OK onwards with our first project


In [ ]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to


In [19]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown in english."

In [20]:
# A function that writes a User Prompt that asks for summaries of websites:


def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [21]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)


In [22]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"},
]

In [23]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me put on my thinking cap for this one. The answer is 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function


In [24]:
# See how this function creates exactly the format above


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]

In [25]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown in english.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it

## Time to bring it together - the API for OpenAI is very simple!


In [26]:
# And now: call the OpenAI API. You will get very familiar with this!


def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
summarize("https://edwarddonner.com")

'# Website Summary: Edward Donner\n\nThe **Edward Donner** website is a personal platform showcasing Ed’s interests and professional endeavors, particularly his work with Large Language Models (LLMs). Ed identifies as a coder and LLM experimenter and shares his background as the co-founder and CTO of **Nebula.io**, a company focused on leveraging AI to enhance talent discovery and management. Previously, Ed founded **untapt**, an AI startup that was acquired in 2021. \n\n## Key Features:\n\n- **Interests**: Ed also engages in DJing and amateur electronic music production.\n- **Professional Focus**: The website highlights his work with proprietary LLMs and innovative approaches to talent matching.\n- **Connect Four**: An interactive platform where LLMs compete in strategic gameplay.\n\n## News and Announcements:\n\n1. **September 15, 2025**: Discussion on "AI in Production: Gen AI and Agentic AI on AWS at scale".\n2. **May 28, 2025**: Announcement of his courses aimed at training future

In [28]:
# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
display_summary("https://cnn.com")

# CNN Overview

CNN is a comprehensive news platform that provides the latest updates, in-depth analyses, and coverage of global events across various categories including US news, world affairs, politics, business, health, entertainment, sports, science, and climate. 

## Current Highlights:

- **Government Shutdown**: There are growing concerns as Congress faces a deadline to avert a government shutdown, with the chances looking slim.
  
- **Middle East Conflict**: Discussions are ongoing regarding Trump's peace plan for Gaza and the current complexities surrounding a ceasefire.

- **Recent Accidents**: Reports detail a church shooting in Michigan where all are accounted for, alongside ongoing investigations.

- **Entertainment News**: Nicole Kidman and Keith Urban have announced their separation.

- **Climate Events**: Namibia has deployed its army to combat a severe wildfire affecting Etosha game reserve. 

- **Consumer Alerts**: McDonald’s announces a return of the Monopoly promotion with a modern twist.

- **Health Studies**: Recent findings indicate a drop in abortions provided by clinicians in the U.S. during the first half of 2025.

CNN strives to engage its audience with real-time updates, insightful analysis, and a range of multimedia content including videos and interactive segments.